In [59]:
%pip install memory-profiler
%load_ext memory_profiler

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [60]:
#! import all necessary lib
from PIL import Image
import numpy as np
import csv
import sys
from memory_profiler import profile

# CONVERT MAP RECEIVE TO BINARY MATRIX

In [61]:

image = Image.open("../rawData/Map19_12.pgm")
image.save("../map/Map19_12.png")

In [62]:
#! convert map image to matrix binary


image_path = "../map/Map19_12.png"
image = Image.open(image_path).convert("RGB")

image_arr = np.array(image)

def convertToMatrix(image_arr):
    matrix = np.zeros((image_arr.shape[0], image_arr.shape[1]), dtype=int)
    
    #! map white pixels to 255
    whiteMask = (image_arr == [255,255,255]).all(axis=-1) | (image_arr == [254,254,254]).all(axis = -1)
    matrix[whiteMask] = 1
    
    return matrix

matrix = convertToMatrix(image_arr)
outputCsvFile ="../shortestPathResult/dataCSV/map1912.csv"

with open(outputCsvFile, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(matrix)
print(f"matrix have saved in path: {outputCsvFile}")


matrix have saved in path: ../shortestPathResult/dataCSV/map1912.csv


In [63]:


# Load the matrix from a CSV file
def load_matrix_from_csv(file_path):
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        matrix = []
        for row in reader:
            # Convert numeric values and leave 'X' as is
            matrix.append([int(cell) if cell.isdigit() else cell for cell in row])
    return matrix

# Convert the matrix to a PNG image
def matrix_to_png(matrix, output_file):
    height = len(matrix)
    width = len(matrix[0])

    # Create a new RGB image
    image = Image.new("RGB", (width, height))

    # Map matrix values to RGB
    pixels = image.load()
    for y in range(height):
        for x in range(width):
            if matrix[y][x] == 0:  # Obstacle
                pixels[x, y] = (0, 0, 0)  # Black
            elif matrix[y][x] == 1:  # Free space
                pixels[x, y] = (255, 255, 255)  # White
            elif matrix[y][x] == 'X':  # Robot path
                pixels[x, y] = (255, 255, 0)  # Yellow

    # Save the image
    image.save(output_file)
    print(f"PNG file saved as '{output_file}'")

# Main function
def convertFromCSVToPNG(yourInputCSVPath: str,yourAlgor: str):
    # Path to your CSV file
    input_csv = yourInputCSVPath

    # Output PNG file
    output_png = "../shortestPathResult/"+yourAlgor+".png"

    # Load the matrix and convert it to PNG
    matrix = load_matrix_from_csv(input_csv)
    matrix_to_png(matrix, output_png)




# PREPROCESS DATA

In [64]:

def loadMatrix(path):
    with open(path, mode="r") as file:
        reader = csv.reader(file)
        matrix = [list(map(int, row)) for row in reader]
    return matrix
def markPath(matrix, path):
    temp = matrix
    for (x,y) in path:
        temp[x][y] = "X"
    return temp
def getPoint(matrix):
    rows, cols = len(matrix), len(matrix[0])
    travelPoint = [(i,j) for i in range(rows) for j in range(cols) if matrix[i][j] == 1]
    if not travelPoint:
        return None, None
    
    def distance(p1,p2):
        import math
        return math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2 )
    
    start = min(travelPoint, key=lambda p: distance((0,0),p))
    end = max(travelPoint, key=lambda p: distance((0,0),p))
    return start,end
def saveResult(matrix, path):
    temp = matrix
    with open(path, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerows(temp)
    print(f"Result has been saved to path: {path}")

# DFS

In [65]:
def DFS(matrixOriginal, start: set, end: set):
    matrix = matrixOriginal
    rows, cols = len(matrix), len(matrix[0])
    stack = [start]
    visited = set()
    parent = {} #! for truy vet the path
    print(f"All memory have use: {sys.getsizeof(stack) + sys.getsizeof(visited)+sys.getsizeof(parent)} bytes")
    while stack:
        x,y = stack.pop()
        if (x,y) == end: #! termination
            path = []
            while (x,y) in parent:
                path.append((x,y))
                x,y = parent[(x,y)]
                
            path.append(start)
            return path[::-1]
        
        #! move robot
        if (x,y) not in visited:
            visited.add((x,y))
            
            for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
                nx, ny = x + dx, y+dy
                if (0 <= nx < rows and 0 <= ny < cols and matrix[nx][ny] == 1 and (nx,ny) not in visited):
                    stack.append((nx,ny))
                    parent[(nx,ny)] = (x,y)
    
    
                    
    #! termination for no path found
    return None

# BFS

In [66]:

def BFS(matrixOriginal, start, end):
    matrix = matrixOriginal
    from collections import deque
    rows, cols = len(matrix), len(matrix[0])
    queue = deque()
    queue.append(start)
    visited = set()
    parent = {}
    print(f"All memory have use: {sys.getsizeof(queue) + sys.getsizeof(visited)+sys.getsizeof(parent)} bytes")
    while queue:
        x,y = queue[0]
        queue.popleft()
        if (x,y) == end:
            path = []
            while(x,y) in parent:
                path.append((x,y))   
                x,y = parent[(x,y)]
            path.append(start)
            return path[::-1]
        if (x,y) not in visited:
            visited.add((x,y))
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < rows and 0 <= ny < cols and matrix[nx][ny] == 1 and (nx, ny) not in visited:
                    queue.append((nx, ny))
                    parent[(nx, ny)] = (x, y)  # Track the parent for path reconstruction
    return None

# Astar Algorithm

In [67]:

def aStar(matrixOriginal, start: set, end: set):
    matrix = matrixOriginal
    import heapq #! working as priority queue
    rows, cols = len(matrix), len(matrix[0])
    
    openList = []
    heapq.heappush(openList,(0,start)) #! (f, (x,y))
    gCost = {start: 0}
    parent = {}
    print(f"All memory have use: {sys.getsizeof(openList)+sys.getsizeof(parent) + sys.getsizeof(gCost)} bytes")
    # def heuristic(a,b):
    #     '''
    #     return mahatan distance
    #     '''
    #     return abs(a[0] - b[0]) + abs(a[1] - b[1])
    def heuristic(a,b):
        import math
        return math.sqrt((a[0]-b[0])**2 + (a[1] - b[1])**2)

    while openList:
        _, current = heapq.heappop(openList)
        x,y = current
        
        if current == end:
            path = []
            while current in parent:
                path.append(current)
                current = parent[current]
            path.append(start)
            return path[::-1]
    
        for dx, dy in [(-1,0), (1,0), (0,-1),(0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < rows and 0 <= ny < cols and matrix[nx][ny] == 1:
                #print(f"current start ({nx},{ny})")
                gNew = gCost[(x,y)] + 1
                if (nx, ny) not in gCost or gNew < gCost[(nx, ny)]:
                    gCost[(nx,ny)] = gNew
                    fCost = gNew + heuristic((nx,ny), end)
                    heapq.heappush(openList, (fCost, (nx,ny)))
                    parent[(nx,ny)] = (x,y)
    return None

# Hill Climbing algorithm

In [68]:

def HillClimb(matrixOriginal, start,end):
    matrix = matrixOriginal
    rows, cols = len(matrix), len(matrix[0])
    
    def heuristic(p1,p2):
        #print(f"this is p1: {p1}")
        #print(f"this is p2 {p2}")
        return abs(p2[0] - p1[0]) + abs(p2[1] - p1[1])
    visited = [[False for _ in range(cols)] for _ in range(rows)]
    curr = start
    if isinstance(curr,int):
        raise ValueError("Start should be a set of tuple (x,y)")
    currPath = [curr]
    visited[curr[0]][curr[1]] = True
    print(f"All memory have use: {sys.getsizeof(visited) + sys.getsizeof(currPath)} bytes")
    
    while(curr != end):
        x,y = curr
        
            
        neighbour = []
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = x+dx, y+dy
            if 0 <= nx < rows and 0 <= ny < cols and matrix[nx][ny] == 1:
                neighbour.append((heuristic((nx,ny), end),(nx,ny)))
        if not neighbour:
            print("stuck at local optimum")
            return currPath
        neighbour.sort(key=lambda x: x[0])
        bestNeighbour = neighbour[0][1]
        
        if heuristic(bestNeighbour, end) >= heuristic(curr,end):
            print("cant find better neighbour, stuck at local minimum")
            return currPath
        
        curr = bestNeighbour
        currPath.append(curr)
        visited[curr[0]][curr[1]] = True
    return currPath

# Genetic algorithm

In [69]:

import random
random.seed(1234)

def Genetic(matrixOriginal, start, end, populationSize=100, generations=200, mutationRate=0.1):
    matrix = matrixOriginal
    rows, cols = len(matrix), len(matrix[0])
    
    def heuristic(p1, p2):
        return abs(p2[0] - p1[0]) + abs(p2[1] - p1[1])
    
    def generateIndividual():
        path = [start]
        while(path[-1] != end):
            x, y = path[-1]
            neighbour = [(x + dx, y + dy) for dx, dy in [(-1, 0), (0, -1), (1, 0), (0, 1)] 
                        if 0 <= x + dx < rows and 0 <= y + dy < cols and matrix[x + dx][y + dy] == 1]
            if not neighbour:
                break  # no valid neighbour, then stop
            
            path.append(random.choice(neighbour))
        return path
    
    def fitness(individual):
        '''
        Calculate fitness based on path length and reaching the goal.
        '''
        if individual[-1] != end:
            return float('inf')
        return len(individual) + heuristic(individual[-1], end)
    
    def crossOver(parent1, parent2):
        # Ensure both parents are long enough for crossover
        min_length = min(len(parent1), len(parent2))
        
        # Avoid crossover if either parent is too short
        if min_length <= 1:
            return parent1  # Or return a modified version if desired
        
        split = random.randint(1, min_length - 1)  # This ensures the range is valid
        child = parent1[:split]
        
        for point in parent2:
            if point not in child:
                child.append(point)
            if point == end:
                break
        return child
    
    def mutate(individual):
        '''
        Choose a random neighbour that valid for going
        '''
        if random.random() < mutationRate:
            # Check if the individual is long enough to mutate
            if len(individual) > 2:  # Ensure the length is greater than 2 to mutate
                idx = random.randint(1, len(individual) - 2)  # Avoid mutating start or end
                x, y = individual[idx]
                neighbour = [(x + dx, y + dy) for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]
                            if 0 <= x + dx < rows and 0 <= y + dy < cols and matrix[x + dx][y + dy] == 1]
                if neighbour:
                    individual[idx] = random.choice(neighbour)
        return individual

    # Initialize the population
    population = [generateIndividual() for _ in range(populationSize)]
    print(f"Memory usage: {sys.getsizeof(population)} bytes")
    for generation in range(generations):
        population.sort(key=fitness)
        if fitness(population[0]) == len(population[0]):
            print(f"Solution found at generation {generation}")
            return population[0]
        
        # Selection: Keep the top 50%
        topIndividual = population[:populationSize // 2]
        
        # Crossover and mutation
        newPopulation = []
        for _ in range(populationSize):
            parent1, parent2 = random.sample(topIndividual, k=2)  # Fix: specify k=2 for selection
            print(f"Parents before cross over: {parent1, parent2}")
            child = crossOver(parent1, parent2)
            print(f"child after crossOver: {child}")
            child = mutate(child)
            print(f"Child after mutate: {child}")
            newPopulation.append(child)
        
        population = newPopulation
    
    print("No solution found!")
    return population[0]


# UNIFORM COST SEARCH

In [70]:
import heapq

def ucs(matrix, startPoint, endPoint):
    """
    Uniform Cost Search (UCS) algorithm for a grid-based matrix.
    
    Args:
        matrix (list of list of int): The map where 1 = walkable, 0 = obstacle.
        startPoint (tuple): The starting point as (row, col).
        endPoint (tuple): The ending point as (row, col).
    
    Returns:
        list: The shortest path from startPoint to endPoint as a list of (row, col) tuples.
              Returns an empty list if no path exists.
    """
    rows, cols = len(matrix), len(matrix[0])
    
    # Check if startPoint or endPoint are unreachable
    if matrix[startPoint[0]][startPoint[1]] == 0 or matrix[endPoint[0]][endPoint[1]] == 0:
        return []  # Start or end is blocked
    
    # Handle trivial case where start and end are the same
    if startPoint == endPoint:
        return [startPoint]
    
    # Priority queue for UCS (cost, (row, col), path)
    queue = [(0, startPoint, [startPoint])]
    visited = set()  # To keep track of visited nodes
    
    # Directions for moving up, down, left, right
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    directions_memory = sys.getsizeof(directions)
    visited_memory = sys.getsizeof(visited)
    queue_memory = sys.getsizeof(queue)
    
    print(f"Memory usage: {directions_memory + visited_memory + queue_memory} bytes")
    
    
    while queue:
        # Get the node with the smallest cost
        cost, (row, col), path = heapq.heappop(queue)
        
        # If we reached the goal, return the path
        if (row, col) == endPoint:
            return path
        
        # Skip if the cell is already visited
        if (row, col) in visited:
            continue
        visited.add((row, col))
        
        # Explore neighbors
        for dr, dc in directions:
            new_row, new_col = row + dr, col + dc
            
            # Check bounds and if the cell is walkable
            if 0 <= new_row < rows and 0 <= new_col < cols and matrix[new_row][new_col] == 1:
                if (new_row, new_col) not in visited:
                    # Add neighbor to the queue with updated cost
                    heapq.heappush(queue, (cost + 1, (new_row, new_col), path + [(new_row, new_col)]))
    
    # If we exhaust the queue without finding the goal, return an empty list
    return []


In [71]:
def ids(matrix, startPoint, endPoint):
    """
    Iterative Deepening Search (IDS) algorithm for a grid-based matrix.
    
    Args:
        matrix (list of list of int): The map where 1 = walkable, 0 = obstacle.
        startPoint (tuple): The starting point as (row, col).
        endPoint (tuple): The ending point as (row, col).
    
    Returns:
        list: The shortest path from startPoint to endPoint as a list of (row, col) tuples.
              Returns an empty list if no path exists.
    """
    rows, cols = len(matrix), len(matrix[0])

    def dls(node, depth, path, visited):
        """
        Depth-Limited Search (DLS) helper function.
        
        Args:
            node (tuple): Current position as (row, col).
            depth (int): Current depth limit.
            path (list): Current path.
            visited (set): Set of visited nodes.
        
        Returns:
            list: Path to the goal if found, else None.
        """
        if depth < 0:
            return None
        
        row, col = node
        
        # If the node is the goal, return the path
        if node == endPoint:
            return path
        
        # Mark the node as visited
        visited.add(node)
        
        # Explore neighbors
        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            new_row, new_col = row + dr, col + dc
            neighbor = (new_row, new_col)
            
            # Check bounds and if the neighbor is walkable and not visited
            if (0 <= new_row < rows and 0 <= new_col < cols and
                    matrix[new_row][new_col] == 1 and neighbor not in visited):
                # Recur with a reduced depth
                result = dls(neighbor, depth - 1, path + [neighbor], visited)
                if result:
                    return result
        
        # Unmark the node before returning (backtrack)
        visited.remove(node)
        return None

    # Perform IDS with increasing depth
    for depth in range(rows * cols):
        visited = set()
        result = dls(startPoint, depth, [startPoint], visited)
        if result:
            return result
    
    # If no path is found
    return []


In [72]:
import numpy as np
import random

def train_q_learning(matrix, startPoint, endPoint, episodes=1000, alpha=0.1, gamma=0.9, epsilon=0.1):
    """
    Q-learning algorithm to train an agent to navigate a grid.
    
    Args:
        matrix (list of list of int): The map where 1 = walkable, 0 = obstacle.
        startPoint (tuple): The starting point as (row, col).
        endPoint (tuple): The ending point as (row, col).
        episodes (int): Number of training episodes.
        alpha (float): Learning rate.
        gamma (float): Discount factor.
        epsilon (float): Exploration rate.
    
    Returns:
        np.ndarray: Trained Q-table.
    """
    rows, cols = len(matrix), len(matrix[0])
    actions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right
    
    # Initialize Q-table
    q_table = np.zeros((rows, cols, len(actions)))
    
    for _ in range(episodes):
        # Reset environment
        state = startPoint
        
        while state != endPoint:
            row, col = state
            
            # Choose an action using epsilon-greedy strategy
            if random.uniform(0, 1) < epsilon:
                action_idx = random.choice(range(len(actions)))  # Explore
            else:
                action_idx = np.argmax(q_table[row, col])  # Exploit
            
            # Perform the action
            dr, dc = actions[action_idx]
            new_row, new_col = row + dr, col + dc
            
            # Check bounds and if the move is valid
            if 0 <= new_row < rows and 0 <= new_col < cols and matrix[new_row][new_col] == 1:
                next_state = (new_row, new_col)
            else:
                next_state = state  # Invalid move stays in the same state
            
            # Calculate reward
            if next_state == endPoint:
                reward = 100  # High reward for reaching the goal
            elif next_state == state:
                reward = -1  # Penalty for hitting walls or invalid moves
            else:
                reward = -0.1  # Small penalty for each move
            
            # Update Q-value using the Q-learning formula
            old_value = q_table[row, col, action_idx]
            next_max = np.max(q_table[new_row, new_col])
            q_table[row, col, action_idx] = old_value + alpha * (reward + gamma * next_max - old_value)
            
            # Transition to the next state
            state = next_state
    
    return q_table


def find_path_q_learning(matrix, startPoint, endPoint, q_table):
    """
    Use the trained Q-table to find the optimal path.
    
    Args:
        matrix (list of list of int): The map where 1 = walkable, 0 = obstacle.
        startPoint (tuple): The starting point as (row, col).
        endPoint (tuple): The ending point as (row, col).
        q_table (np.ndarray): Trained Q-table.
    
    Returns:
        list: The optimal path from startPoint to endPoint as a list of (row, col) tuples.
    """
    state = startPoint
    path = [state]
    
    while state != endPoint:
        row, col = state
        action_idx = np.argmax(q_table[row, col])
        dr, dc = [(-1, 0), (1, 0), (0, -1), (0, 1)][action_idx]
        new_row, new_col = row + dr, col + dc
        
        # Check bounds and valid moves
        if 0 <= new_row < len(matrix) and 0 <= new_col < len(matrix[0]) and matrix[new_row][new_col] == 1:
            state = (new_row, new_col)
        else:
            break  # Exit if stuck in an invalid move
        
        path.append(state)
    
    return path if state == endPoint else []


# Example usage
def rl(grid, start,end):
    q_table = train_q_learning(grid, start, end)
    path = find_path_q_learning(grid, start, end, q_table)
    print("Optimal Path:", path)


# MAIN FUNCTION

In [73]:
#! main function to run all algorithm
MAPPATH = "../shortestPathResult/dataCSV/map1912.csv"
matrix = loadMatrix(MAPPATH)

In [74]:
allAlgorName = ["DFS", "BFS", "aStar", "HillClimb", "Genetic", "ucs", "ids", "rl"]
theEndPath = "path.csv"
savePath = "../shortestPathResult/"
allResultPath = [savePath + allAlgorName[i] + theEndPath for i in range(len(allAlgorName))]


In [75]:
allResultPath

['../shortestPathResult/DFSpath.csv',
 '../shortestPathResult/BFSpath.csv',
 '../shortestPathResult/aStarpath.csv',
 '../shortestPathResult/HillClimbpath.csv',
 '../shortestPathResult/Geneticpath.csv',
 '../shortestPathResult/ucspath.csv',
 '../shortestPathResult/idspath.csv',
 '../shortestPathResult/rlpath.csv']

In [76]:
import time
start, end = getPoint(matrix)
print(f"start point: {start}")
print(f"end point: {end}")

for i in range(len(allResultPath)):
    if allAlgorName[i] == "DFS":
        MAPPATH = "../shortestPathResult/dataCSV/map1912.csv"
        matrix = loadMatrix(MAPPATH)
        startTime = time.time()
        #DFS(matrix,start,end)
        
        path = DFS(matrix,start,end)
        endTime = time.time()
        print(f"{allAlgorName[i]} takes {endTime - startTime} seconds to run")
        if path:
            print(f"Path found for {allAlgorName[i]} algorithm!: ")
            print(path)
            matrixWithPath = markPath(matrix, path)
            
            saveResult(matrixWithPath, allResultPath[i])
            convertFromCSVToPNG(allResultPath[i],allAlgorName[i])
        else:
            print(f"No path exist between {start} to {end} for algorithm {allAlgorName[i]}")
    if allAlgorName[i] == "BFS":
        MAPPATH = "../shortestPathResult/dataCSV/map1912.csv"
        matrix = loadMatrix(MAPPATH)
        startTime = time.time()
        path = BFS(matrix,start,end)
        endTime = time.time()
        print(f"{allAlgorName[i]} takes {endTime - startTime} seconds to run")
        if path:
            print(f"Path found for {allAlgorName[i]} algorithm!: ")
            print(path)
            matrixWithPath = markPath(matrix, path)
            
            saveResult(matrixWithPath, allResultPath[i])
            convertFromCSVToPNG(allResultPath[i],allAlgorName[i])
        else:
            print(f"No path exist between {start} to {end} for algorithm {allAlgorName[i]}")
    if allAlgorName[i] == "aStar":
        MAPPATH = "../shortestPathResult/dataCSV/map1912.csv"
        matrix = loadMatrix(MAPPATH)
        startTime = time.time()
        path = aStar(matrix,start,end)
        endTime = time.time()
        print(f"{allAlgorName[i]} takes {endTime - startTime} seconds to run")
        if path:
            print(f"Path found for {allAlgorName[i]} algorithm!: ")
            print(path)
            matrixWithPath = markPath(matrix, path)
            
            saveResult(matrixWithPath, allResultPath[i])
            convertFromCSVToPNG(allResultPath[i],allAlgorName[i])
        else:
            print(f"No path exist between {start} to {end} for algorithm {allAlgorName[i]}")
    if allAlgorName[i] == "ucs":
        MAPPATH = "../shortestPathResult/dataCSV/map1912.csv"
        matrix = loadMatrix(MAPPATH)
        startTime = time.time()
        path = ucs(matrix,start,end)
        endTime = time.time()
        print(f"{allAlgorName[i]} takes {endTime - startTime} seconds to run")
        if path:
            print(f"Path found for {allAlgorName[i]} algorithm!: ")
            print(path)
            matrixWithPath = markPath(matrix, path)
            
            saveResult(matrixWithPath, allResultPath[i])
            convertFromCSVToPNG(allResultPath[i],allAlgorName[i])
        else:
            print(f"No path exist between {start} to {end} for algorithm {allAlgorName[i]}")
    # if allAlgorName[i] == "ids":
    #     MAPPATH = "../shortestPathResult/dataCSV/map1912.csv"
    #     matrix = loadMatrix(MAPPATH)
    #     path = ids(matrix,start,end)
    #     if path:
    #         print(f"Path found for {allAlgorName[i]} algorithm!: ")
    #         print(path)
    #         matrixWithPath = markPath(matrix, path)
            
    #         saveResult(matrixWithPath, allResultPath[i])
    #         convertFromCSVToPNG(allResultPath[i],allAlgorName[i])
    #     else:
    #         print(f"No path exist between {start} to {end} for algorithm {allAlgorName[i]}")
    
    if allAlgorName[i] == "Genetic":
        MAPPATH = "../shortestPathResult/dataCSV/map1912.csv"
        matrix = loadMatrix(MAPPATH)
        startTime = time.time()
        path = Genetic(matrix,start,end)
        endTime = time.time()
        print(f"{allAlgorName[i]} takes {endTime - startTime} seconds to run")
        if path:
            print(f"Path found for {allAlgorName[i]} algorithm!: ")
            print(path)
            matrixWithPath = markPath(matrix, path)
            
            saveResult(matrixWithPath, allResultPath[i])
            convertFromCSVToPNG(allResultPath[i],allAlgorName[i])
        else:
            print(f"No path exist between {start} to {end} for algorithm {allAlgorName[i]}")
    if allAlgorName[i] == "HillClimb":
        MAPPATH = "../shortestPathResult/dataCSV/map1912.csv"
        matrix = loadMatrix(MAPPATH)
        startTime = time.time()
        path = HillClimb(matrix,start,end)
        endTime = time.time()
        print(f"{allAlgorName[i]} takes {endTime - startTime} seconds to run")
        if path:
            print(f"Path found for {allAlgorName[i]} algorithm!: ")
            print(path)
            matrixWithPath = markPath(matrix, path)
            
            saveResult(matrixWithPath, allResultPath[i])
            convertFromCSVToPNG(allResultPath[i],allAlgorName[i])
        else:
            print(f"No path exist between {start} to {end} for algorithm {allAlgorName[i]}")
    # if allAlgorName[i] == "rl":
    #     MAPPATH = "../shortestPathResult/dataCSV/map1912.csv"
    #     matrix = loadMatrix(MAPPATH)
    #     path = rl(matrix,start,end)
    #     if path:
    #         print(f"Path found for {allAlgorName[i]} algorithm!: ")
    #         print(path)
    #         matrixWithPath = markPath(matrix, path)
            
    #         saveResult(matrixWithPath, allResultPath[i])
    #         convertFromCSVToPNG(allResultPath[i],allAlgorName[i])
    #     else:
    #         print(f"No path exist between {start} to {end} for algorithm {allAlgorName[i]}")
    

start point: (26, 14)
end point: (75, 101)
All memory have use: 344 bytes
DFS takes 0.006916046142578125 seconds to run
Path found for DFS algorithm!: 
[(26, 14), (26, 15), (26, 16), (26, 17), (26, 18), (26, 19), (26, 20), (26, 21), (26, 22), (26, 23), (26, 24), (26, 25), (26, 26), (26, 27), (26, 28), (26, 29), (26, 30), (26, 31), (26, 32), (26, 33), (26, 34), (26, 35), (26, 36), (26, 37), (26, 38), (26, 39), (26, 40), (26, 41), (26, 42), (26, 43), (26, 44), (26, 45), (26, 46), (26, 47), (26, 48), (26, 49), (26, 50), (26, 51), (26, 52), (26, 53), (26, 54), (26, 55), (26, 56), (26, 57), (26, 58), (26, 59), (26, 60), (26, 61), (26, 62), (26, 63), (26, 64), (26, 65), (26, 66), (26, 67), (26, 68), (26, 69), (26, 70), (27, 70), (27, 69), (27, 68), (27, 67), (27, 66), (27, 65), (27, 64), (27, 63), (27, 62), (27, 61), (27, 60), (27, 59), (27, 58), (27, 57), (27, 56), (27, 55), (27, 54), (27, 53), (27, 52), (27, 51), (27, 50), (27, 49), (27, 48), (27, 47), (27, 46), (27, 45), (27, 44), (27, 43

Result has been saved to path: ../shortestPathResult/DFSpath.csv
PNG file saved as '../shortestPathResult/DFS.png'
All memory have use: 1040 bytes
BFS takes 0.006006479263305664 seconds to run
Path found for BFS algorithm!: 
[(26, 14), (26, 15), (26, 16), (26, 17), (26, 18), (26, 19), (26, 20), (26, 21), (26, 22), (26, 23), (26, 24), (26, 25), (26, 26), (26, 27), (26, 28), (26, 29), (26, 30), (26, 31), (26, 32), (26, 33), (26, 34), (26, 35), (26, 36), (26, 37), (26, 38), (26, 39), (26, 40), (26, 41), (26, 42), (26, 43), (26, 44), (26, 45), (26, 46), (26, 47), (26, 48), (26, 49), (26, 50), (26, 51), (26, 52), (26, 53), (26, 54), (26, 55), (26, 56), (26, 57), (26, 58), (26, 59), (26, 60), (26, 61), (26, 62), (26, 63), (26, 64), (26, 65), (27, 65), (28, 65), (29, 65), (30, 65), (31, 65), (32, 65), (33, 65), (34, 65), (35, 65), (36, 65), (37, 65), (38, 65), (39, 65), (40, 65), (40, 66), (40, 67), (40, 68), (41, 68), (41, 69), (41, 70), (42, 70), (42, 71), (43, 71), (44, 71), (45, 71), (46,